<a href="https://colab.research.google.com/github/mertserbes/mertserbes_case/blob/main/FinancialSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Hasan Taskın 704241004
#Mert Serbes 504241538

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_google_news(keywords=["BIST100", "Borsa İstanbul", "Hisse"], limit=100):
    news_data = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    print(f"Veri çekiliyor: {keywords}")

    for keyword in keywords:
        # Google News RSS URL'si (Türkçe Haberler)
        url = f"https://news.google.com/rss/search?q={keyword}&hl=tr&gl=TR&ceid=TR:tr"

        try:
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.content, features="xml") # XML parser kullanıyoruz

            items = soup.find_all("item")

            print(f"'{keyword}' için {len(items)} haber bulundu.")

            for item in items:
                title = item.title.text
                pub_date = item.pubDate.text
                link = item.link.text
                source = item.source.text if item.source else "Bilinmiyor"

                # Sadece benzersiz başlıkları al
                if not any(d['Title'] == title for d in news_data):
                    news_data.append({
                        "Date": pub_date,
                        "Title": title,
                        "Source": source,
                        "Keyword": keyword,
                        "Label": "" # Burayı sen dolduracaksın
                    })

                if len(news_data) >= limit:
                    break

            time.sleep(1) # Nezaketen bekleme

        except Exception as e:
            print(f"Hata: {e}")

    df = pd.DataFrame(news_data)
    return df

# --- ÇALIŞTIR ---
# 300-500 arası haber toplamaya çalış
df_news = scrape_google_news(keywords=["BIST100", "Borsa", "Ekonomi", "Dolar TL", "Faiz Kararı"], limit=500)

print(f"Toplam {len(df_news)} satır veri çekildi.")
print(df_news.head())

# CSV Olarak Kaydet
df_news.to_csv("bist100_sentiment_dataset.csv", index=False)

In [ ]:
!pip install newspaper3k feedparser lxml_html_clean

In [12]:
import feedparser
from newspaper import Article
import pandas as pd
from tqdm import tqdm
from urllib.parse import quote  # <-- YENİ EKLENEN KISIM: URL Kodlayıcı

def scrape_full_content(keywords=["BIST100", "Borsa", "Hisse"], limit_per_key=50):
    data_list = []

    # Kullanıcı gibi görünmek için User-Agent
    config_headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    for keyword in keywords:
        print(f"\n--- '{keyword}' için linkler toplanıyor ---")

        # --- DÜZELTME BURADA ---
        # Kelimedeki boşlukları ve Türkçe karakterleri URL uyumlu hale getiriyoruz
        # Örn: "Borsa İstanbul" -> "Borsa%20%C4%B0stanbul" olur.
        encoded_keyword = quote(keyword)

        rss_url = f"https://news.google.com/rss/search?q={encoded_keyword}&hl=tr&gl=TR&ceid=TR:tr"
        # -----------------------

        feed = feedparser.parse(rss_url)

        print(f"{len(feed.entries)} adet link bulundu. İçerikler indiriliyor...")

        count = 0
        for entry in tqdm(feed.entries):
            if count >= limit_per_key:
                break

            try:
                url = entry.link

                # Newspaper3k ile Haberin İçine Gir
                article = Article(url)
                article.download()
                article.parse()

                # Çok kısa metinleri atla
                if len(article.text) < 50:
                    continue

                data_list.append({
                    "Date": entry.published,
                    "Title": article.title,
                    "Text": article.text,
                    "Source": entry.source.title if hasattr(entry, 'source') else "Google News",
                    "URL": url
                })

                count += 1

            except Exception as e:
                # Bazı linkler hata verebilir, devam et
                continue

    return pd.DataFrame(data_list)

# --- ÇALIŞTIR ---
# Artık içinde boşluk olan kelimeler ("Borsa İstanbul" gibi) hata vermez.
df_full = scrape_full_content(keywords=["BIST100", "Borsa İstanbul", "Ekonomi"], limit_per_key=30)

# Sonuçları Gör
print(f"\nToplam {len(df_full)} satır veri çekildi.")
print(df_full.head())

# Kaydet
df_full.to_csv("bist100_full_text_dataset.csv", index=False)


--- 'BIST100' için linkler toplanıyor ---
100 adet link bulundu. İçerikler indiriliyor...


100%|██████████| 100/100 [00:47<00:00,  2.12it/s]



--- 'Borsa İstanbul' için linkler toplanıyor ---
100 adet link bulundu. İçerikler indiriliyor...


100%|██████████| 100/100 [00:44<00:00,  2.24it/s]



--- 'Ekonomi' için linkler toplanıyor ---
102 adet link bulundu. İçerikler indiriliyor...


100%|██████████| 102/102 [00:46<00:00,  2.20it/s]


Toplam 0 satır veri çekildi.
Empty DataFrame
Columns: []
Index: []
